In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd
import numpy as np
import re
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Import train_test_split and preprocessing libraries / classes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Import ML models for regression problems
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# Import ML models for classification problems
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Import model performace metric libraries
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score

# Import for optimization
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [3]:
# Load your dataset
df = pd.read_csv('/Users/marina/Desktop/final_project/data/cleaned/okcupid_scaledfinal.csv')


In [4]:

columns_to_exclude = [col for col in df.columns if col.startswith('city_') or col.startswith('sign_')] + ['income', 'height']
df_filtered = df.drop(columns=columns_to_exclude)

In [6]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Extract features and profile IDs
X = df_filtered.drop(['profile_id'], axis=1).values
profile_ids = df_filtered['profile_id'].values

# Initialize and fit the KNN model
knn = NearestNeighbors(n_neighbors=20, algorithm='auto')
knn.fit(X)

# Function to find 20 most similar profiles
def find_similar_profiles(profile_index):
    distances, indices = knn.kneighbors([X[profile_index]])
    similar_profiles = profile_ids[indices[0]]
    return similar_profiles

# Example: Find 20 most similar profiles to the first profile in the dataset
similar_profiles = find_similar_profiles(0)
print("20 most similar profiles to the first profile:", similar_profiles)

20 most similar profiles to the first profile: [ 1000 14269 18153 40328 39676 40579 23058 17999 42702 23438 15244 15826
 40853 35169 31046 44321 42766 34366 33946  2617]


In [6]:
from sklearn.metrics import pairwise_distances

# Calculate pairwise distances between the first profile and all other profiles
distances = pairwise_distances([X[0]], X, metric='euclidean')[0]
sorted_indices = np.argsort(distances)[1:21]  # Exclude the first one (distance to itself)
similar_profiles = profile_ids[sorted_indices]

print("20 most similar profiles to the first profile:", similar_profiles)

20 most similar profiles to the first profile: [14269 18153 40328 39676 40579 23058 17999 42702 23438 15244 15826 40853
 35169 31046 44321 42766 34366 33946  2617 23640]


In [8]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score

# Assuming df is your DataFrame with excluded columns
columns_to_exclude = [col for col in df.columns if col.startswith('city_') or col.startswith('sign_')] + ['income', 'height']
df_filtered = df.drop(columns=columns_to_exclude)

# Extract features and profile IDs
X = df_filtered.drop(['profile_id'], axis=1).values
profile_ids = df_filtered['profile_id'].values

# Initialize K-Means clustering
kmeans = KMeans(n_clusters=10, random_state=42)  # Adjust number of clusters as needed
cluster_labels = kmeans.fit_predict(X)

# Function to find similar profiles based on cluster
def find_similar_profiles_in_cluster(profile_id, cluster_labels, profile_ids):
    target_index = np.where(profile_ids == profile_id)[0][0]
    target_cluster = cluster_labels[target_index]
    cluster_indices = np.where(cluster_labels == target_cluster)[0]
    similar_profiles = profile_ids[cluster_indices]
    return similar_profiles

# Example: Find similar profiles to the first profile in the dataset
similar_profiles = find_similar_profiles_in_cluster(profile_ids[0], cluster_labels, profile_ids)
print("Similar profiles based on clustering:", similar_profiles)

# Evaluate clustering performance
silhouette_avg = silhouette_score(X, cluster_labels)
print(f'Silhouette Score: {silhouette_avg}')


Similar profiles based on clustering: [ 1000  1004  1006 ... 48172 48210 48220]
Silhouette Score: 0.03025933843387327


In [10]:
# Example: Try different numbers of clusters
for n_clusters in range(2, 20):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(f'Number of clusters: {n_clusters}, Silhouette Score: {silhouette_avg}')

Number of clusters: 2, Silhouette Score: 0.060801009559868426
Number of clusters: 3, Silhouette Score: 0.045815149879220265
Number of clusters: 4, Silhouette Score: 0.049138102394876586
Number of clusters: 5, Silhouette Score: 0.041978230660278414
Number of clusters: 6, Silhouette Score: 0.03669376199007406
Number of clusters: 7, Silhouette Score: 0.0322502413192531
Number of clusters: 8, Silhouette Score: 0.033196393591706966
Number of clusters: 9, Silhouette Score: 0.030427796972497343
Number of clusters: 10, Silhouette Score: 0.03025933843387327
Number of clusters: 11, Silhouette Score: 0.028584813911189753
Number of clusters: 12, Silhouette Score: 0.02702499853924773
Number of clusters: 13, Silhouette Score: 0.02350972624110045
Number of clusters: 14, Silhouette Score: 0.026316933077493432
Number of clusters: 15, Silhouette Score: 0.02546149153745833
Number of clusters: 16, Silhouette Score: 0.0226080568172662


KeyboardInterrupt: 

In [11]:
from sklearn.decomposition import PCA

# Example: Apply PCA before clustering
pca = PCA(n_components=10)  # Adjust number of components based on explained variance
X_pca = pca.fit_transform(X)

kmeans = KMeans(n_clusters=10, random_state=42)
cluster_labels = kmeans.fit_predict(X_pca)
silhouette_avg = silhouette_score(X_pca, cluster_labels)
print(f'Silhouette Score after PCA: {silhouette_avg}')


Silhouette Score after PCA: 0.10532438812866918


In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
import numpy as np
from scipy.stats import mode
from sklearn.metrics import silhouette_score

# Assuming X and profile_ids are defined as before

# Initialize individual clustering algorithms
kmeans = KMeans(n_clusters=10, random_state=42)
agg_clustering = AgglomerativeClustering(n_clusters=10)

# Fit individual clustering models
kmeans_labels = kmeans.fit_predict(X)
agg_labels = agg_clustering.fit_predict(X)

# Ensemble by averaging
ensemble_labels = np.mean([kmeans_labels, agg_labels], axis=0).astype(int)

# Evaluate the ensemble clustering
silhouette_avg_ensemble = silhouette_score(X, ensemble_labels)
print(f'Silhouette Score with Ensemble Clustering (Averaging): {silhouette_avg_ensemble}')
